In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
sys.path.append(os.path.abspath('../src'))
from portfolio_utils import load_asset_data, compute_daily_returns

# Load processed CSVs
tsla, spy, bnd = load_asset_data(
    "../data/processed/TSLA_clean.csv",
    "../data/processed/SPY_clean.csv",
    "../data/processed/BND_clean.csv"
)

# Compute daily returns
tsla_returns = compute_daily_returns(tsla)
spy_returns = compute_daily_returns(spy)
bnd_returns = compute_daily_returns(bnd)

# Combine into one DataFrame
returns_df = pd.concat([tsla_returns, spy_returns, bnd_returns], axis=1)
returns_df.columns = ["TSLA", "SPY", "BND"]

returns_df.head()


,TSLA,SPY,BND
Date,,,
2015-07-02,0.040386,-0.000916,0.002472
2015-07-06,-0.001071,-0.002846,0.003452
2015-07-07,-0.042328,0.006289,0.000983
2015-07-08,-0.048231,-0.016777,0.002825
2015-07-09,0.011610,0.001809,-0.003306
